In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import os
import time
import gc
import re
import random

#设置随机种子保证可重复性
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything()

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
print("Train shape : ", train.shape)
print("Test shape : ", test.shape)

In [ ]:
%%time
train["question_text"] = train["question_text"].str.lower()
test["question_text"] = test["question_text"].str.lower()

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):

    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x


train["question_text"] = train["question_text"].apply(lambda x: clean_text(x))
test["question_text"] = test["question_text"].apply(lambda x: clean_text(x))

In [ ]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = None # how many unique words to use (i.e num rows in embedding vector)
maxlen = 72 # max number of words in a question to use #99.99%

## fill up the missing values
X = train["question_text"].fillna("_na_").values
X_test = test["question_text"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features, filters='')
tokenizer.fit_on_texts(list(X)+list(X_test))

X = tokenizer.texts_to_sequences(X)
X_test = tokenizer.texts_to_sequences(X_test)

## Pad the sentences 
X = pad_sequences(X, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

## Get the target values
Y = train['target'].values

sub = test[['qid']]

In [ ]:
del train, test
gc.collect()

In [ ]:
word_index = tokenizer.word_index
max_features = len(word_index)+1
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) 
                            for o in open(EMBEDDING_FILE) 
                            if o.split(" ")[0] in word_index or o.split(" ")[0].lower() in word_index)

    emb_mean, emb_std = -0.005838499, 0.48782197

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        elif embeddings_index.get(word.lower()) is not None:
            embedding_matrix[i] = embeddings_index.get(word.lower())
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) 
                            for o in open(EMBEDDING_FILE) 
                            if len(o)>100 and o.split(" ")[0] in word_index )

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) 
                            for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') 
                            if len(o)>100 and (o.split(" ")[0] in word_index or o.split(" ")[0].lower() in word_index))

    emb_mean, emb_std = -0.0053247833, 0.49346462
    
    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        elif embeddings_index.get(word.lower()) is not None:
            embedding_matrix[i] = embeddings_index.get(word.lower())
    
    return embedding_matrix

In [ ]:
%%time
seed_everything()
embedding_matrix_1 = load_glove(word_index)
embedding_matrix_3 = load_para(word_index)
embedding_matrix = np.concatenate((embedding_matrix_1, embedding_matrix_3), axis=1)  
del embedding_matrix_1, embedding_matrix_3
gc.collect()
np.shape(embedding_matrix)

In [ ]:
class AdamW(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/4)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.wd = K.variable(weight_decay, name='weight_decay') # decoupled weight decay (2/4)
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd # decoupled weight decay (3/4)

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - lr * wd * p # decoupled weight decay (4/4)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
embed_size = 600
maxlen = 72

In [ ]:
def GRU_pool():
    K.clear_session()       
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(rate=0.22, seed=1008600)(x)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True, 
                                kernel_initializer=glorot_normal(seed=1008600), 
                               recurrent_initializer=orthogonal(gain=1.0, seed=1008600)))(x)

    x1 = GlobalMaxPool1D()(x)
    x2 = GlobalAvgPool1D()(x)
    concat = concatenate([x1, x2], axis=-1)
    x = Dense(1, activation="sigmoid", kernel_initializer=glorot_normal(seed=1008600))(concat)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer=AdamW(weight_decay=0.06),)
    return model

In [ ]:
#epoch=5
def parallelRNN():
    K.clear_session()
    recurrent_units = 128
    inp = Input(shape=(maxlen,))
    embed = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    embed = SpatialDropout1D(0.2, seed=1008600)(embed)

    x = Bidirectional(CuDNNGRU(64, return_sequences=True, 
                                   kernel_initializer=glorot_uniform(seed=1008600), 
                                   recurrent_initializer=Orthogonal(gain=1.0, seed=1008600)))(embed)
    y = Bidirectional(CuDNNLSTM(64, return_sequences=True,
                                  kernel_initializer=glorot_uniform(seed=1008600), 
                                  recurrent_initializer=Orthogonal(gain=1.0, seed=1008600)))(embed)
    concat = concatenate([x, y], axis=-1)
    concat = GlobalMaxPooling1D()(concat)


    out = Dense(1, activation="sigmoid", kernel_initializer=glorot_uniform(seed=1008600))(concat)
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='binary_crossentropy', optimizer=AdamW(weight_decay=0.06))
    return model

In [ ]:
def stackGRU():
    K.clear_session()       
    inp = Input(shape=(maxlen,))
    
    embed = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False, name='Embedding')(inp)
    embed = SpatialDropout1D(0.22, seed=11110000)(embed)

    rnn1 = Bidirectional(CuDNNGRU(64, return_sequences=True, kernel_initializer=glorot_uniform(seed=111100), 
                           recurrent_initializer=Orthogonal(gain=1.0, seed=123000)))(embed)
    rnn2 = Bidirectional(CuDNNGRU(64, return_sequences=True, kernel_initializer=glorot_uniform(seed=111000), 
                           recurrent_initializer=Orthogonal(gain=1.0, seed=1203000)))(rnn1)

    x = concatenate([rnn1, rnn2])
    x = GlobalMaxPooling1D()(x)  
    out = Dense(1, activation='sigmoid', kernel_initializer=glorot_uniform(seed=111100))(x)
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='binary_crossentropy', optimizer=AdamW(weight_decay=0.06),)
    return model

In [ ]:
def f1_smart(y_true, y_pred):
    args = np.argsort(y_pred)
    tp = y_true.sum()
    fs = (tp - np.cumsum(y_true[args[:-1]])) / np.arange(y_true.shape[0] + tp - 1, tp, -1)
    res_idx = np.argmax(fs)
    return 2 * fs[res_idx], (y_pred[args[res_idx]] + y_pred[args[res_idx + 1]]) / 2

In [17]:
def lr_scheduler(epoch, lr=0.001):
    if epoch == 4:
        lr = 0.0002
    return lr

In [ ]:
%%time
seed_everything()
kfold = StratifiedKFold(n_splits=7, random_state=10, shuffle=True)
bestscore = []
logloss = []
y_test = np.zeros((X_test.shape[0], ))
oof = np.zeros((X.shape[0], ))

for i, (train_index, valid_index) in enumerate(kfold.split(X, Y)):
    print('Fold %s'%(i+1))
    X_train, X_val, Y_train, Y_val = X[train_index], X[valid_index], Y[train_index], Y[valid_index]
    filepath="weights_best.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=2, save_best_only=True, mode='min')
    #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=1, min_lr=0.0001, verbose=2)
    #earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=2, mode='auto')
    scheduler = LearningRateScheduler(lr_scheduler, verbose=1)
    callbacks = [checkpoint, scheduler]
    if i == 0:
        model = stackGRU()
    elif i == 1:
        model = GRU_pool()
    elif i == 2:
        model = parallelRNN()
    print(model.summary())
    model.fit(X_train, Y_train, batch_size=512, epochs=6, validation_data=(X_val, Y_val), verbose=2, 
              callbacks=callbacks, 
              shuffle=False,
              class_weight={0:1, 1:1.25}
             )
    model.load_weights(filepath)
    y_pred = model.predict([X_val], batch_size=1024, verbose=2)
    y_test += np.squeeze(model.predict([X_test], batch_size=1024, verbose=2))/3
    oof[valid_index] = np.squeeze(y_pred)
    f1, threshold = f1_smart(np.squeeze(Y_val), np.squeeze(y_pred))
    print('Optimal F1: {:.4f} at threshold: {:.4f}'.format(f1, threshold))
    bestscore.append(threshold)
    logloss.append(np.min(model.history.history['val_loss']))
    print('*'*50)
    print('\n')
    if i == 2:
        break

In [ ]:
np.mean(logloss), np.mean(bestscore)

In [ ]:
y_test = y_test.reshape((-1, 1))
pred_test_y = (y_test>np.mean(bestscore)).astype(int)
sub['prediction'] = pred_test_y
sub.to_csv("submission.csv", index=False)